In [13]:
%load_ext autoreload
%autoreload 2
!hostname
!pwd
import os, sys
print(sys.executable)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
fish-gcp005
/home/akarshkumar0101/nca-alife
/home/akarshkumar0101/miniconda3/envs/nca-alife-jax/bin/python


In [14]:
import os, sys, glob, pickle
from functools import partial

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
# sns.set_theme()

from tqdm.auto import tqdm
from einops import rearrange, reduce, repeat

In [15]:
# import experiment_utils
import time
import subprocess

# Launch Experiment (on this node)

In [20]:
header = '\n'.join([
    '#!/bin/bash',
    
    # 'source ~/.virtualenvs/nca-alife-torch/bin/activate'
    
    'source ~/miniconda3/etc/profile.d/conda.sh',
    'conda activate nca-alife-jax',
    
    'cd ~/nca-alife/src'
])


gpus = [4, 5, 6, 7]
log_dir = os.path.expanduser(f'~/nca-alife-data/logs/{time.strftime("%Y_%m_%d_%H_%M_%S")}')
exp_sh = f'./src/experiments/main.sh'
with open(exp_sh, 'r') as f:
    commands = [c for c in f.read().split('\n') if c]
print(f"{exp_sh} has {len(commands)} commands: ")
print('\n'.join(commands[:3]), '\n...')
print(f"Logging to {log_dir}")

./src/experiments/main.sh has 56 commands: 
python train_lenia_evosax.py --seed=0 --save_dir="/home/akarshkumar0101/nca-alife-data/lenia_prompt1/0_Sep_CMA_ES_0" --prompt="a biological cell"                      --algo="Sep_CMA_ES" --rollout_steps=200 --bs=32 --n_iters=10000
python train_lenia_evosax.py --seed=0 --save_dir="/home/akarshkumar0101/nca-alife-data/lenia_prompt1/0_Sep_CMA_ES_1" --prompt="a biological cell under the microscope" --algo="Sep_CMA_ES" --rollout_steps=200 --bs=32 --n_iters=10000
python train_lenia_evosax.py --seed=0 --save_dir="/home/akarshkumar0101/nca-alife-data/lenia_prompt1/0_Sep_CMA_ES_2" --prompt="a red organism"                         --algo="Sep_CMA_ES" --rollout_steps=200 --bs=32 --n_iters=10000 
...
Logging to /home/akarshkumar0101/nca-alife-data/logs/2024_08_06_09_06_00


In [21]:
print(log_dir)

/home/akarshkumar0101/nca-alife-data/logs/2024_08_06_09_06_00


In [22]:
!mkdir -p {log_dir}
!rm {os.path.dirname(log_dir)+'/latest'}
!ln -sf {log_dir} {os.path.dirname(log_dir)+'/latest'}
for i_gpu, gpu in enumerate(gpus):
    job_ids = np.arange(len(commands))[i_gpu::len(gpus)]
    cvd = f'export CUDA_VISIBLE_DEVICES={gpu}'
    txt = f"{header}\n{cvd}\n\n"
    for job_id in job_ids:
        txt += f"touch {log_dir}/job_{job_id:05d}.status\n"
        txt += f"{commands[job_id]} &> {log_dir}/job_{job_id:05d}.out\n"
        txt += f"echo $? > {log_dir}/job_{job_id:05d}.status\n\n"
    with open(f"{log_dir}/gpu_{gpu}.sh", "w") as f:
        f.write(txt)

In [23]:
print('\n'.join(txt.split('\n')[:24]))

#!/bin/bash
source ~/miniconda3/etc/profile.d/conda.sh
conda activate nca-alife-jax
cd ~/nca-alife/src
export CUDA_VISIBLE_DEVICES=7

touch /home/akarshkumar0101/nca-alife-data/logs/2024_08_06_09_06_00/job_00003.status
python train_lenia_evosax.py --seed=0 --save_dir="/home/akarshkumar0101/nca-alife-data/lenia_prompt1/0_Sep_CMA_ES_3" --prompt="a cool colorful pattern"                --algo="Sep_CMA_ES" --rollout_steps=200 --bs=32 --n_iters=10000 &> /home/akarshkumar0101/nca-alife-data/logs/2024_08_06_09_06_00/job_00003.out
echo $? > /home/akarshkumar0101/nca-alife-data/logs/2024_08_06_09_06_00/job_00003.status

touch /home/akarshkumar0101/nca-alife-data/logs/2024_08_06_09_06_00/job_00007.status
python train_lenia_evosax.py --seed=1 --save_dir="/home/akarshkumar0101/nca-alife-data/lenia_prompt1/1_Sep_CMA_ES_0" --prompt="a biological cell"                      --algo="Sep_CMA_ES" --rollout_steps=200 --bs=32 --n_iters=10000 &> /home/akarshkumar0101/nca-alife-data/logs/2024_08_06_09_06_00/

In [24]:
for i_gpu, gpu in enumerate(gpus):
    command = f"bash {log_dir}/gpu_{gpu}.sh"
    print(command)
    process = subprocess.Popen(command, shell=True)

bash /home/akarshkumar0101/nca-alife-data/logs/2024_08_06_09_06_00/gpu_4.sh
bash /home/akarshkumar0101/nca-alife-data/logs/2024_08_06_09_06_00/gpu_5.sh
bash /home/akarshkumar0101/nca-alife-data/logs/2024_08_06_09_06_00/gpu_6.sh
bash /home/akarshkumar0101/nca-alife-data/logs/2024_08_06_09_06_00/gpu_7.sh


# Track Experiment

In [22]:
import subprocess
a = subprocess.run(['ps', 'ux'], stdout=subprocess.PIPE).stdout.decode('utf-8')

In [25]:
[l for l in a.split('\n') if "bash" in l and "gpu" in l]

['akarshk+ 1271851  0.0  0.0   2480   572 ?        S    04:58   0:00 /bin/sh -c bash /home/akarshkumar0101/nca-alife-data/logs/2024_07_31_04_58_39/gpu_4.sh',
 'akarshk+ 1271852  0.0  0.0   2480   516 ?        S    04:58   0:00 /bin/sh -c bash /home/akarshkumar0101/nca-alife-data/logs/2024_07_31_04_58_39/gpu_5.sh',
 'akarshk+ 1271853  0.0  0.0 219972  3408 ?        S    04:58   0:00 bash /home/akarshkumar0101/nca-alife-data/logs/2024_07_31_04_58_39/gpu_4.sh',
 'akarshk+ 1271854  0.0  0.0   2480   512 ?        S    04:58   0:00 /bin/sh -c bash /home/akarshkumar0101/nca-alife-data/logs/2024_07_31_04_58_39/gpu_6.sh',
 'akarshk+ 1271855  0.0  0.0 219972  3200 ?        S    04:58   0:00 bash /home/akarshkumar0101/nca-alife-data/logs/2024_07_31_04_58_39/gpu_5.sh',
 'akarshk+ 1271856  0.0  0.0   2480   516 ?        S    04:58   0:00 /bin/sh -c bash /home/akarshkumar0101/nca-alife-data/logs/2024_07_31_04_58_39/gpu_7.sh',
 'akarshk+ 1271857  0.0  0.0 219972  3460 ?        S    04:58   0:00 bash 

Terminated
Terminated
Terminated
Terminated


In [41]:
from collections import defaultdict

statuses = []
for job_id in range(len(commands)):
    try:
        with open(f"{log_dir}/job_{job_id:05d}.status", "r") as f:
            status = int(f.read().strip())
            statuses.append(status)
    except Exception as e:
        pass
        # print(e)
        # break
statuses = np.array(statuses)

In [42]:
(statuses==0).sum(), len(commands)

(49, 56)

In [43]:
np.unique(statuses, return_counts=True)

(array([0, 1]), array([49,  7]))

In [45]:
statuses.argmax()

11

In [49]:
commands[11]

'python train_lenia_evosax.py --seed=1 --save_dir="/home/akarshkumar0101/nca-alife-data/lenia_prompt1/1_Sep_CMA_ES_4" --prompt="a bacterium under the microscope"       --algo="Sep_CMA_ES" --rollout_steps=200 --bs=32 --n_iters=10000'

In [50]:
log_dir

'/home/akarshkumar0101/nca-alife-data/logs/2024_08_06_09_06_00'

In [40]:
for job_id in list(range(len(commands))):
    print()
    print(f'------------------------------------------------------ {job_id} ------------------------------------------------------')
    print()
    try:
        with open(f"{log_dir}/job_{job_id:05d}.out", "r") as f:
            print(f.readlines()[-1].strip())
    except Exception as e:
        # pass
        break
        print(e)


------------------------------------------------------ 0 ------------------------------------------------------

100%|██████████| 10000/10000 [15:19<00:00, 10.87it/s, loss=-0.312]

------------------------------------------------------ 1 ------------------------------------------------------

100%|██████████| 10000/10000 [18:00<00:00,  9.26it/s, loss=-0.296]

------------------------------------------------------ 2 ------------------------------------------------------

100%|██████████| 10000/10000 [17:41<00:00,  9.42it/s, loss=-0.311]

------------------------------------------------------ 3 ------------------------------------------------------

100%|██████████| 10000/10000 [17:51<00:00,  9.33it/s, loss=-0.335]

------------------------------------------------------ 4 ------------------------------------------------------

100%|██████████| 10000/10000 [15:23<00:00, 10.83it/s, loss=-0.302]

------------------------------------------------------ 5 -------------------------------------

In [14]:
log_dir

'/home/akarshkumar0101/nca-alife-data/logs/2024_08_02_05_16_15'